<a href="https://colab.research.google.com/github/Vaishnavilande/ML-Projects-Machine-Learning-/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [3]:
train = pd.read_csv("/content/train (1).csv")
test = pd.read_csv("/content/test (1).csv")
     

In [4]:
train.shape

(891, 12)

In [5]:
test.shape

(418, 11)

In [7]:
#missing value
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
#missing value
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [19]:
train['Age'].fillna(train['Age'].mean(), inplace=True)
test['Age'].fillna(test['Age'].mean(), inplace=True)


In [20]:
train['Cabin'] = train['Cabin'].fillna(train['Cabin'].value_counts().index[0])

In [21]:
test['Cabin'] = test['Cabin'].fillna(test['Cabin'].value_counts().index[0])

In [22]:
#missing value
test.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [23]:
#missing value
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [24]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,B96 B98,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,B96 B98,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,B96 B98,S


In [25]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,B57 B59 B63 B66,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,B57 B59 B63 B66,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,B57 B59 B63 B66,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,B57 B59 B63 B66,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,B57 B59 B63 B66,S


In [26]:
#columns with object type values
train.select_dtypes('object').columns

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

In [28]:
{column: len(train[column].unique()) for column in train.select_dtypes('object').columns}

{'Name': 891, 'Sex': 2, 'Ticket': 681, 'Cabin': 147, 'Embarked': 3}

In [29]:
{column: list(train[column].unique()) for column in train.select_dtypes('object').columns}

{'Name': ['Braund, Mr. Owen Harris',
  'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
  'Heikkinen, Miss. Laina',
  'Futrelle, Mrs. Jacques Heath (Lily May Peel)',
  'Allen, Mr. William Henry',
  'Moran, Mr. James',
  'McCarthy, Mr. Timothy J',
  'Palsson, Master. Gosta Leonard',
  'Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)',
  'Nasser, Mrs. Nicholas (Adele Achem)',
  'Sandstrom, Miss. Marguerite Rut',
  'Bonnell, Miss. Elizabeth',
  'Saundercock, Mr. William Henry',
  'Andersson, Mr. Anders Johan',
  'Vestrom, Miss. Hulda Amanda Adolfina',
  'Hewlett, Mrs. (Mary D Kingcome) ',
  'Rice, Master. Eugene',
  'Williams, Mr. Charles Eugene',
  'Vander Planke, Mrs. Julius (Emelia Maria Vandemoortele)',
  'Masselmani, Mrs. Fatima',
  'Fynney, Mr. Joseph J',
  'Beesley, Mr. Lawrence',
  'McGowan, Miss. Anna "Annie"',
  'Sloper, Mr. William Thompson',
  'Palsson, Miss. Torborg Danira',
  'Asplund, Mrs. Carl Oscar (Selma Augusta Emilia Johansson)',
  'Emir, Mr. Farred Chehab',
  

In [30]:
#create function for gender encoding 

def encode_sex(x):
    if x.lower()[0] == 'f':   #if first letter of gender ie 0 is equal to f 
        return 0
    elif x.lower()[0] == 'm':
        return 1
    else:
        return 2

In [33]:
#apply the above function
train['Sex'] = train['Sex'].apply(encode_sex)

In [34]:
nominal_features = [
    'Name',
    'Ticket', 
    'Cabin', 
    'Embarked'
   
]

In [42]:
def onehot_encode(data, columns, prefixes):
    data = data.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(train[column], prefix)
        data = pd.concat([data, dummies], axis=1)
        data = data.drop(column, axis=1)
    return data

In [43]:
nominal_prefixes = [
     'Na',
    'Ti', 
    'Ca', 
    'Em'
    
]

In [44]:
train = onehot_encode(
    train,
    columns=nominal_features,
    prefixes=nominal_prefixes
)

In [45]:
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,"Na_Abbing, Mr. Anthony","Na_Abbott, Mr. Rossmore Edward",...,Ca_F G73,Ca_F2,Ca_F33,Ca_F38,Ca_F4,Ca_G6,Ca_T,Em_C,Em_Q,Em_S
0,1,0,3,1,22.000000,1,0,7.2500,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,1,1,0,38.000000,1,0,71.2833,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,1,3,0,26.000000,0,0,7.9250,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,1,1,0,35.000000,1,0,53.1000,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,0,3,1,35.000000,0,0,8.0500,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.000000,0,0,13.0000,0,0,...,0,0,0,0,0,0,0,0,0,1
887,888,1,1,0,19.000000,0,0,30.0000,0,0,...,0,0,0,0,0,0,0,0,0,1
888,889,0,3,0,29.699118,1,2,23.4500,0,0,...,0,0,0,0,0,0,0,0,0,1
889,890,1,1,1,26.000000,0,0,30.0000,0,0,...,0,0,0,0,0,0,0,1,0,0


In [50]:
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.50000,0,0,330911,7.8292,B57 B59 B63 B66,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.00000,1,0,363272,7.0000,B57 B59 B63 B66,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.00000,0,0,240276,9.6875,B57 B59 B63 B66,Q
3,895,3,"Wirz, Mr. Albert",male,27.00000,0,0,315154,8.6625,B57 B59 B63 B66,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.00000,1,1,3101298,12.2875,B57 B59 B63 B66,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,30.27259,0,0,A.5. 3236,8.0500,B57 B59 B63 B66,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.00000,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.50000,0,0,SOTON/O.Q. 3101262,7.2500,B57 B59 B63 B66,S
416,1308,3,"Ware, Mr. Frederick",male,30.27259,0,0,359309,8.0500,B57 B59 B63 B66,S


In [46]:
#columns with object type values
test.select_dtypes('object').columns

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

In [47]:
{column: len(train[column].unique()) for column in train.select_dtypes('object').columns}

{}

In [49]:
{column: list(train[column].unique()) for column in train.select_dtypes('object').columns}

{}

In [51]:
#create function for gender encoding 

def encode_sex(x):
    if x.lower()[0] == 'f':   #if first letter of gender ie 0 is equal to f 
        return 0
    elif x.lower()[0] == 'm':
        return 1
    else:
        return 2

In [52]:
#apply the above function
test['Sex'] = test['Sex'].apply(encode_sex)

In [53]:
nominal_features = [
    'Name',
    'Ticket', 
    'Cabin', 
    'Embarked'
   
]

In [55]:
def onehot_encode(data, columns, prefixes):
    data = data.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(test[column], prefix)
        data = pd.concat([data, dummies], axis=1)
        data = data.drop(column, axis=1)
    return data

In [56]:
nominal_prefixes = [
     'Na',
    'Ti', 
    'Ca', 
    'Em'
    
]

In [57]:
test = onehot_encode(
    test,
    columns=nominal_features,
    prefixes=nominal_prefixes
)

In [58]:
test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,"Na_Abbott, Master. Eugene Joseph","Na_Abelseth, Miss. Karen Marie","Na_Abelseth, Mr. Olaus Jorgensen",...,Ca_F E46,Ca_F E57,Ca_F G63,Ca_F2,Ca_F33,Ca_F4,Ca_G6,Em_C,Em_Q,Em_S
0,892,3,1,34.50000,0,0,7.8292,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,893,3,0,47.00000,1,0,7.0000,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,894,2,1,62.00000,0,0,9.6875,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,895,3,1,27.00000,0,0,8.6625,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,896,3,0,22.00000,1,1,12.2875,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,1,30.27259,0,0,8.0500,0,0,0,...,0,0,0,0,0,0,0,0,0,1
414,1306,1,0,39.00000,0,0,108.9000,0,0,0,...,0,0,0,0,0,0,0,1,0,0
415,1307,3,1,38.50000,0,0,7.2500,0,0,0,...,0,0,0,0,0,0,0,0,0,1
416,1308,3,1,30.27259,0,0,8.0500,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [59]:
X_train = train.drop('Sex', axis = 1)
Y_train= train['Sex']

In [61]:
X_test = train.drop('Sex', axis = 1)
Y_test= train['Sex']

In [65]:
import statistics
class KNN_Classifier():

  # initiating the parameters
  def __init__(self, distance_metric):

    self.distance_metric = distance_metric

  # getting the distance metric
  def get_distance_metric(self,training_data_point, test_data_point):

    if (self.distance_metric == 'euclidean'):

      dist = 0
      for i in range(len(training_data_point) - 1):
        dist = dist + (training_data_point[i] - test_data_point[i])**2

      euclidean_dist = np.sqrt(dist)
    
      return euclidean_dist

    elif (self.distance_metric == 'manhattan'):

      dist = 0

      for i in range(len(training_data_point) - 1):
        dist = dist + abs(training_data_point[i] - test_data_point[i])

      manhattan_dist = dist

      return manhattan_dist

  # getting the nearest neighbors
  def nearest_neighbors(self,X_train, test_data, k):

    distance_list = []

    for training_data in X_train:

      distance = self.get_distance_metric(training_data, test_data)
      distance_list.append((training_data, distance))

    distance_list.sort(key=lambda x: x[1])

    neighbors_list = []

    for j in range(k):
      neighbors_list.append(distance_list[j][0])

    return neighbors_list


  # predict the class of the new data point:
  def predict(self,X_train, test_data, k):
    neighbors = self.nearest_neighbors(X_train, test_data, k)
    
    for data in neighbors:
      label = []
      label.append(data[-1])

    predicted_class = statistics.mode(label)

    return predicted_class

In [63]:
classifier = KNN_Classifier(distance_metric='manhattan')